In [1]:
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [3]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [4]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20465
Most common words [('the', 10129), ('a', 7312), ('and', 6199), ('of', 6063), ('to', 4233), ('is', 3378)]
Sample data [4, 666, 9, 2834, 8, 22, 4, 3605, 13927, 98] ['the', 'rock', 'is', 'destined', 'to', 'be', 'the', '21st', 'centurys', 'new']


In [5]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [6]:
class Model:
    def __init__(self, size_layer, embedded_size, maxlen,
                batch_size, dict_size, dimension_output, grad_clip=5.0):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.GRUCell(size_layer,kernel_initializer=tf.orthogonal_initializer(),
                                          reuse=reuse)
        
        self.X = tf.placeholder(tf.int32, [None, maxlen])
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        enc_rnn_out, enc_state = tf.nn.dynamic_rnn(cell = cells(), 
                                                   inputs = encoder_embedded,
                                                   dtype = tf.float32)
        def loop_fn(state, masks):
            query = tf.expand_dims(state, -1)
            align = tf.squeeze(tf.matmul(enc_rnn_out, query), -1)
            return (align * masks)
        
        def point(idx):
            idx = tf.expand_dims(idx, 1)
            b = tf.expand_dims(tf.range(batch_size),1)
            c = tf.concat((tf.to_int64(b), idx), 1)
            return tf.gather_nd(encoder_embedded, c)
        
        starts = tf.fill([batch_size], GO)
        inp = tf.nn.embedding_lookup(encoder_embeddings, starts)
        masks = tf.to_float(tf.sign(self.X))
        outputs = []
        cell = cells()
        for i in range(maxlen):
            _, state = cell(inp, enc_state)
            output = loop_fn(state, masks)
            outputs.append(output)
            idx = tf.argmax(output, -1)
            inp = point(idx)
        outputs = tf.stack(outputs, 1)
        self.logits = tf.layers.dense(outputs,dimension_output)[:,-1]
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        params = tf.trainable_variables()
        gradients = tf.gradients(self.cost, params)
        clipped_gradients, _ = tf.clip_by_global_norm(gradients, grad_clip)
        self.optimizer = tf.train.AdamOptimizer().apply_gradients(zip(clipped_gradients, params))
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [7]:
size_layer = 128
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128

In [8]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,embedded_size,maxlen,
              batch_size,vocabulary_size+4,dimension_output)
sess.run(tf.global_variables_initializer())

In [9]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.492188
time taken: 11.403055429458618
epoch: 0, training loss: 0.812980, training acc: 0.504261, valid loss: 0.729203, valid acc: 0.492188

epoch: 1, pass acc: 0.492188, current acc: 0.494141
time taken: 10.927583456039429
epoch: 1, training loss: 0.647366, training acc: 0.604048, valid loss: 0.719738, valid acc: 0.494141

epoch: 2, pass acc: 0.494141, current acc: 0.495605
time taken: 11.873044490814209
epoch: 2, training loss: 0.556725, training acc: 0.751894, valid loss: 0.750480, valid acc: 0.495605

epoch: 3, pass acc: 0.495605, current acc: 0.499023
time taken: 26.222250938415527
epoch: 3, training loss: 0.408614, training acc: 0.856534, valid loss: 0.855179, valid acc: 0.499023

epoch: 4, pass acc: 0.499023, current acc: 0.501953
time taken: 30.78032660484314
epoch: 4, training loss: 0.238832, training acc: 0.928622, valid loss: 1.072518, valid acc: 0.501953

time taken: 31.080484867095947
epoch: 5, training loss: 0.119922, training a